# 3. Radial Basis Function Network Network

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

## Initialize and load data

In [108]:
import numpy as np
import xarray as xr
import xarray.ufuncs as xrf
import matplotlib.pyplot as plt
from tqdm.contrib.itertools import product

from system_identification.utils.vdom import tabulate
from system_identification.rbfnn_model import RadialBasisFunctionNeuralNetworkModel

In [3]:
data = xr.open_dataset("data_smoothed.nc")
data

<xarray.Dataset>
Dimensions:                 (t: 10001, dim_0: 4, dim_1: 4, x_idx: 4, training_samples: 8000, inputs: 2, m: 1, outputs: 1, validation_samples: 2001, d0: 2, d1: 2)
Coordinates:
  * t                       (t) float64 0.0 0.01 0.02 0.03 ... 99.98 99.99 100.0
Dimensions without coordinates: dim_0, dim_1, x_idx, training_samples, inputs, m, outputs, validation_samples, d0, d1
Data variables: (12/40)
    p_k1k1                  (t, dim_0, dim_1) float64 ...
    p_kk1                   (t, dim_0, dim_1) float64 ...
    phi                     (t, dim_0, dim_1) float64 ...
    gamma                   (t, dim_0, dim_1) float64 ...
    ps                      (t, dim_0, dim_1) float64 ...
    x_k1k1                  (t, x_idx) float64 ...
    ...                      ...
    alpha_estimate          (t) float64 ...
    training_inputs         (training_samples, inputs, m) float64 ...
    training_outputs        (training_samples, outputs, m) float64 ...
    validation_inputs       (validation_samples, inputs, m) float64 ...
    validation_outputs      (validation_samples, outputs, m) float64 ...
    input_range             (d0, d1) float64 ...

## 3.1 Linear Regression
The reconstructed F-16 dataset is approximated using a RBFNN. The RBF centers are placed in a uniform grid along the input range and amplitudes are estimated using Ordinary Least Squares (OLS).

### Hyperparameter optimization
The number of RBF and widths need to be selected, 

In [119]:
rbf_per_side_list = list(range(1, 20))
rbf_width_list = np.linspace(0.5, 50, 30)

shape = (len(rbf_per_side_list), len(rbf_width_list))
error_training_mean = np.empty(shape)
error_training_abs_mean = np.empty(shape)
error_training_jb = np.empty(shape)
error_training_jbp = np.empty(shape)
error_validation_mean = np.empty(shape)
error_validation_abs_mean = np.empty(shape)
error_validation_jb = np.empty(shape)
error_validation_jbp = np.empty(shape)

for (i, rbf_per_side), (j, rbf_width) in product(list(enumerate(rbf_per_side_list)), list(enumerate(rbf_width_list))):
    model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
        n_inputs=2,
        grid_size=[rbf_per_side, rbf_per_side],
        input_range=data.input_range.values,
        rbf_width=rbf_width,
        rbf_amplitude=1,
    )

    model.train(
        inputs=data.training_inputs.values,
        reference_outputs=data.training_outputs.values,
        validation_inputs=data.validation_inputs.values,
        validation_outputs=data.validation_outputs.values,
        method="trainlsqr",
    )
    
    error_training_abs_mean[i, j] = abs(model.training_log.error_training_data).mean()
    error_training_mean[i, j] = model.training_log.error_training_data.mean()
    error_training_jb[i, j] = model.training_log.error_training_jb.item()
    error_training_jbp[i, j] = model.training_log.error_training_jbp.item()
    
    error_validation_abs_mean[i, j] = abs(model.training_log.error_validation_data).mean()
    error_validation_mean[i, j] = model.training_log.error_validation_data.mean()
    error_validation_jb[i, j] = model.training_log.error_validation_jb.item()
    error_validation_jbp[i, j] = model.training_log.error_validation_jbp.item()
    

  0%|          | 0/570 [00:00<?, ?it/s]

In [192]:
print(np.where(error_training_abs_mean == error_training_abs_mean.min()))
print(np.where(error_validation_abs_mean == error_validation_abs_mean.min()))

(array([18]), array([5]))
(array([13]), array([4]))
(array([18]), array([6]))
(array([12]), array([10]))


In [202]:
tabulate({
    "rbf_per_side": rbf_per_side_list[18],
    "rbf_width": rbf_width_list[6],
    "error_training_abs_mean": error_training_abs_mean[10, 6]
}, caption="Selected hyperparameters")

rbf_per_side,19.0
rbf_width,10.741379310344827
error_training_abs_mean,0.001490178486283987


In [201]:
fig, axs = plt.subplots(8, 1, figsize=(8, 12))

# axs[0].set_yscale("log")
axs[0].plot(rbf_width_list, error_training_mean.T)
axs[0].plot(rbf_width_list, error_validation_mean.T)

axs[1].set_yscale("log")
axs[1].plot(rbf_width_list, error_training_abs_mean.T)
axs[1].plot(rbf_width_list, error_validation_abs_mean.T)

axs[2].set_yscale("log")
axs[2].plot(rbf_width_list, error_training_jb.T)
axs[2].plot(rbf_width_list, error_validation_jb.T)

axs[3].plot(rbf_width_list, error_training_jbp.T)
axs[3].plot(rbf_width_list, error_validation_jbp.T)


axs[4].plot(rbf_per_side_list, error_training_mean)
axs[4].plot(rbf_per_side_list, error_validation_mean)

axs[5].set_yscale("log")
axs[5].plot(rbf_per_side_list, error_training_abs_mean)
axs[5].plot(rbf_per_side_list, error_validation_abs_mean)

axs[6].set_yscale("log")
axs[6].plot(rbf_per_side_list, error_training_jb)
axs[6].plot(rbf_per_side_list, error_validation_jb)

axs[7].plot(rbf_per_side_list, error_training_jbp)
axs[7].plot(rbf_per_side_list, error_validation_jbp)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Identification

In [206]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=2,
    grid_size=[10, 10],
    input_range=data.input_range.values,
    rbf_width=9.03448275862069,
    rbf_amplitude=1,
)

model.train(
    inputs=data.training_inputs.values,
    reference_outputs=data.training_outputs.values,
    validation_inputs=data.validation_inputs.values,
    validation_outputs=data.validation_outputs.values,
    method="trainlsqr",
)

### Results

In [207]:
tabulate(
    header=["identification data", "validation data"],
    row_header=['absolute residuals mean', 'residuals mean', "Jarque-Bera", "Jarque-Bera p-value"],
    data=[
        [abs(model.training_log.error_training_data).mean("i").item(),
         abs(model.training_log.error_validation_data).mean("j").item()],
        [model.training_log.error_training_data.mean("i").item(),
         model.training_log.error_validation_data.mean("j").item()],
        [model.training_log.error_training_jb.mean().item(),
         model.training_log.error_validation_jb.mean().item()],
        [model.training_log.error_training_jbp.mean().item(),
         model.training_log.error_validation_jbp.mean().item()]
    ]
)

,identification data,validation data
absolute residuals mean,0.0016050819711778167,0.0016404308722231971
residuals mean,-4.096760982068917e-06,-3.233616271185781e-05
Jarque-Bera,781.7261341445004,100.57210379520812
Jarque-Bera p-value,1.779618003926692e-170,1.4489224587331844e-22


In [208]:
fig = plt.figure()
reference_inputs = data.training_inputs.values
reference_outputs = data.training_outputs.values
output = model.evaluate(reference_inputs)

ax = fig.add_subplot(projection='3d')
ax.scatter(reference_inputs[:, 0, 0], reference_inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(reference_inputs[:, 0, 0], reference_inputs[:, 1, 0], output.squeeze(), s=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 3.2 Levenberg-Marquardt

The reconstructed F-16 dataset is approximated using a RBFNN. The RBF centers are initially placed in a uniform grid along the input range and the weights are given the same value as in the previous section.

The NN is then trained using the Levenberg-Marquardt (LM) learning algorithm to optimize the centers, widths and amplitudes.

### Identification

In [209]:
model = RadialBasisFunctionNeuralNetworkModel.new_grid_placement(
    n_inputs=2,
    grid_size=[10, 10],
    input_range=data.input_range.values,
    rbf_width=9.03448275862069,
    rbf_amplitude=1,
)

In [210]:
model.train(
    inputs=data.training_inputs.values,
    reference_outputs=data.training_outputs.values,
    validation_inputs=data.validation_inputs.values,
    validation_outputs=data.validation_outputs.values,
    epochs=1000,
    goal=1e-6,
    train_log_freq=1,
    method="trainlm",
    mu=10.,
    alpha=0.95
)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [217]:
model.training_log

<xarray.Dataset>
Dimensions:                (epoch: 1000, i: 8000, j: 2001, k: 100, l: 100, m: 2, n: 100, o: 2)
Coordinates:
  * epoch                  (epoch) int64 1 2 3 4 5 6 ... 996 997 998 999 1000
Dimensions without coordinates: i, j, k, l, m, n, o
Data variables: (12/14)
    gradient               (epoch) float64 1.369e+05 3.52e+04 ... 9.016e+03
    error_training_data    (epoch, i) float64 -0.2234 -0.4137 ... -0.01415
    error_validation_data  (epoch, j) float64 -0.2687 -0.2923 ... -0.007894
    error_training_jb      (epoch) float64 800.1 2.127e+04 ... 559.1 299.6
    error_training_jbp     (epoch) float64 1.857e-174 0.0 ... 8.675e-66
    error_training_skew    (epoch) float64 -0.772 -1.844 -1.265 ... 0.452 0.3486
    ...                     ...
    error_validation_jbp   (epoch) float64 2.969e-51 0.0 ... 4.41e-33 1.165e-15
    error_validation_skew  (epoch) float64 -0.8313 -1.994 ... 0.4579 0.3151
    error_validation_kurt  (epoch) float64 3.163 11.1 10.06 ... 2.026 2.346
    weights_a              (epoch, k) float64 0.1426 0.1138 ... 0.6818 0.8911
    weights_c              (epoch, l, m) float64 -0.4377 -0.7251 ... 0.533
    weights_w              (epoch, n, o) float64 9.036 9.037 ... 9.036 9.036

In [228]:
model = RadialBasisFunctionNeuralNetworkModel(
    weights_a=model.training_log.weights_a.sel(epoch=1).values,
    weights_c=model.training_log.weights_c.sel(epoch=1).values,
    weights_w=model.training_log.weights_w.sel(epoch=1).values,
    input_range=model.input_range,
    description="",
)

In [231]:
model

<RadialBasisFunctionNeuralNetwork
 n_inputs=2
 n_hidden=100
 n_outputs=1>

### Results

In [232]:
fig = plt.figure(figsize=(4, 3))
plt.plot(model.weights_c[:, 0], model.weights_c[:, 1], "x")

fig = plt.figure(figsize=(4, 3))
ax = fig.add_subplot(projection='3d')

output = model.evaluate(reference_inputs)
ax.scatter(reference_inputs[:, 0, 0], reference_inputs[:, 1, 0], reference_outputs.squeeze(), s=0.1)
ax.scatter(reference_inputs[:, 0, 0], reference_inputs[:, 1, 0], output.squeeze(), s=0.1)

fig, axs = plt.subplots(4, 1, figsize=(8, 8))
abs(model.training_log.error_training_data).mean("i").plot(ax=axs[0])
model.training_log.error_training_data.mean("i").plot(ax=axs[1])
model.training_log.error_training_jb.plot(ax=axs[2])
model.training_log.error_training_jbp.plot(ax=axs[3])

abs(model.training_log.error_validation_data).mean("j").plot(ax=axs[0])
model.training_log.error_validation_data.mean("j").plot(ax=axs[1])
model.training_log.error_validation_jb.plot(ax=axs[2])
model.training_log.error_validation_jbp.plot(ax=axs[3])

axs[0].set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [216]:
abs(model.training_log.error_training_data.sel(epoch=59)).mean()

<xarray.DataArray 'error_training_data' ()>
array(0.00158973)
Coordinates:
    epoch    int64 59